In [1]:
import numpy as np
import pandas as pd

In [27]:
csv = pd.read_csv('train.txt', sep=" ")

In [28]:
print(len(csv[csv["class"]==1])/len(csv))
print(len(csv[csv["class"]==0])/len(csv))

0.073275
0.926725


In [29]:
csv.head(10)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,class
29652,NaN,NaN,NaN,NaN,NaN,931.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,ELof,szEZ,ZI9m,ib5G6X1eUxUn6,NaN,NaN,0
27644,NaN,NaN,NaN,NaN,NaN,245.0,7.0,NaN,NaN,NaN,...,bTV7qqc,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN,0
29835,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,...,2JfQ3DB,jySVZNlOJy,NaN,ELof,7aLG,RAYp,F2FyR07IdsN7I,am7c,NaN,0
19631,NaN,NaN,NaN,NaN,NaN,1036.0,7.0,NaN,NaN,NaN,...,hHJsvbM,LM8l689qOp,NaN,NaN,Qcbd,6fzt,SbOd7O8ky1wGNxp0Arj0Xs,NaN,NaN,0
13018,NaN,NaN,NaN,NaN,NaN,518.0,7.0,NaN,NaN,NaN,...,APgVoGr,LM8l689qOp,NaN,NaN,kwS7,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,0
36991,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,v5h4IB0,LM8l689qOp,NaN,NaN,szEZ,02N6s8f,F2FyR07IdsN7I,NaN,NaN,0
1326,NaN,NaN,NaN,NaN,NaN,784.0,0.0,NaN,NaN,NaN,...,TIxsTKW,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN,0
42037,NaN,NaN,NaN,NaN,NaN,1393.0,7.0,NaN,NaN,NaN,...,MKETE6Y,jySVZNlOJy,NaN,kG3k,7P5s,RAYp,TCU50_Yjmm6GIBZ0lL_,mj86,NaN,0
42520,NaN,NaN,NaN,NaN,NaN,1127.0,7.0,NaN,NaN,NaN,...,DHPNgqU,LM8l689qOp,NaN,ELof,WqMG,6fzt,SbOd7O8ky1wGNxp0Arj0Xs,am7c,NaN,0
6784,NaN,NaN,NaN,NaN,NaN,630.0,14.0,NaN,NaN,NaN,...,35GRt2K,LM8l689qOp,NaN,ELof,PM2D,RAYp,F2FyR07IdsN7I,am7c,NaN,0


In [58]:
nans = csv.isna().sum()
nans = nans[nans == len(csv)]
print(nans.index.values)
csv = csv.drop(nans.index.values, axis=1)

['Var8' 'Var15' 'Var20' 'Var31' 'Var32' 'Var39' 'Var42' 'Var48' 'Var52'
 'Var55' 'Var79' 'Var141' 'Var167' 'Var169' 'Var175' 'Var185' 'Var209'
 'Var230']


In [75]:
csv.isna().sum()


Var1      39431
Var2      39031
Var3      39032
Var4      38738
Var5      38797
Var6       4397
Var7       4409
Var9      39431
Var10     38797
Var11     39032
Var12     39558
Var13      4409
Var14     39032
Var16     38797
Var17     38738
Var18     38738
Var19     38738
Var21      4397
Var22      4003
Var23     38797
Var24      5761
Var25      4003
Var26     38797
Var27     38797
Var28      4005
Var29     39431
Var30     39431
Var33     39317
Var34     39031
Var35      4003
          ...  
Var200    20291
Var201    29757
Var202        1
Var203      118
Var204        0
Var205     1547
Var206     4397
Var207        0
Var208      118
Var210        0
Var211        0
Var212        0
Var213    39090
Var214    20291
Var215    39439
Var216        0
Var217      581
Var218      581
Var219     4140
Var220        0
Var221        0
Var222        0
Var223     4140
Var224    39362
Var225    20880
Var226        0
Var227        0
Var228        0
Var229    22754
class         0
Length: 213, dtype: int6

In [73]:
csv["Var3"].unique().astype(int)

array([-2147483648,           0,         231,           9,          84,
               744,           3,         180,          27,         633,
              1332,         321,       13662,         189,         228,
               102,         198,         225,        6126,        2895,
               123,        5136,        3987,          18,         129,
               534,         105,          12,          99,         144,
              3168,        1113,          60,        7413,        1242,
              7188,         405,         147,         333,         120,
               969,          36,       19683,        9012,         417,
                21,       10617,          90,         156,        9399,
               138,        1740,          75,        1644,       18066,
                 6,         567,          96,          45,         153,
               126,         165,        4866,         693,          48,
             12633,         600,         540,         498,      

In [92]:
csv_numeric = csv
for col in list(csv):
    if csv[col].dtype == object:
        print(col)
        csv_numeric[col] = pd.to_numeric(csv[col])
csv_numeric
# string_cols = csv[isinstance(csv, str)]

Var191


ValueError: Unable to parse string "r__I" at position 29

In [ ]:
cols = ['StudentID', 'Homework', 'Midterm', 'Project']  # We don't want to convert the Final grade column.
for col in cols:  # Iterate over chosen columns
	grades[col] = pd.to_numeric(grades[col])

In [ ]:
data = csv.values
np.unique(data[~np.isnan(data)])